# MNISTの画像分類

Googleドライブをマウント

In [3]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/My Drive/KDDI関連/Creative.hack/勉強会/MNIST"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/KDDI関連/Creative.hack/勉強会/MNIST


## 必要なライブラリの読み込み

In [8]:
import os
import pathlib

import numpy as np
import pandas as pd 
import tensorflow as tf
import tensorflow.keras.layers as layers

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

## MNISTのダウンロード  
今回は外部のデータセットを使用するが、一般的には自分でデータセットを用意するケースが多い

In [5]:
mnist = tf.keras.datasets.mnist
(X_train, y_train),(X_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


MNISTのデータセットの一部を可視化

In [ ]:
for i in [1,10,100]:
    print("y_train", "(i="+str(i)+"): ", y_train[i])
    print("X_train", "(i="+str(i)+"): ")    
    plt.imshow(X_train[i], cmap='gray')
    plt.show()

## 画像データの正規化
1. min-max normalization（正規化）  
  - 最小値が0, 最大値が1になるように変換する
  - x_new = (x - x_min) / (x_max - x_min)
2. z-score normalization（標準化）
  - 平均が0, 標準偏差が1になるよう変換する
  - x_new = (x - x_mean) / x_std


今回は入力データを標準化して学習を行う。

In [7]:
X_train, X_test = X_train/255.0, X_test/255.0
X_train, X_test = (X_train-0.5)/0.5, (X_test-0.5)/0.5

## モデルの作成
今回は全結合層3層によって構成される簡易的なモデルで学習を行う

In [10]:
class ClassificationModel(tf.keras.Model):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.flatten_layer = layers.Flatten(input_shape=(28, 28), name='input')
        self.linear1 = layers.Dense(512, name='fc_1', activation='relu')
        self.linear2 = layers.Dense(256, name='fc_2', activation='relu')
        self.linear3 = layers.Dense(10, name='fc_2', activation='softmax')
        # self.relu = layers.Activation(tf.nn.relu, name='relu_1')

    def call(self, x):
        x = self.flatten_layer(x)
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        return x

model = ClassificationModel()

In [11]:
model